In [20]:
# -*- coding: UTF-8 -*-
import numpy as np
import pandas as pd
import requests as req

In [21]:
CONFIG = {
    'dep': '../data/deputados-detalhes.csv',
    'prop': '../data/proposicoes.csv'
}

In [22]:
deputados_df = pd.read_csv(CONFIG['dep'])
proposicoes_df = pd.read_csv(CONFIG['prop'])

In [23]:
deputados_df.head()

,anexo,codOrcamento,condicao,email,fone,gabinete,idParlamentar,ideCadastro,matricula,nome,nomeParlamentar,partido,sexo,uf,urlFoto
0,4,3283,Titular,dep.goulart@camara.gov.br,3215-5533,533,5830382,178980,361,ANTONIO GOULART DOS REIS,GOULART,PSD,masculino,SP,http://www.camara.gov.br/internet/deputado/ban...
1,4,2152,Titular,dep.betofaro@camara.gov.br,3215-5723,723,5830714,141335,19,JOSÉ ROBERTO OLIVEIRA FARO,BETO FARO,PT,masculino,PA,http://www.camara.gov.br/internet/deputado/ban...
2,4,2488,Titular,dep.rodrigodecastro@camara.gov.br,3215-5701,701,5830453,141531,268,RODRIGO BATISTA DE CASTRO,RODRIGO DE CASTRO,PSDB,masculino,MG,http://www.camara.gov.br/internet/deputado/ban...
3,4,2551,Titular,dep.laertebessa@camara.gov.br,3215-5340,340,5830785,141478,412,LAERTE RODRIGUES DE BESSA,LAERTE BESSA,PR,masculino,DF,http://www.camara.gov.br/internet/deputado/ban...
4,3,3712,Titular,dep.majorolimpio@camara.gov.br,3215-5279,279,5830405,178826,368,SERGIO OLIMPIO GOMES,MAJOR OLIMPIO,PDT,masculino,SP,http://www.camara.gov.br/internet/deputado/ban...


In [24]:
proposicoes_df.head()

,tipo,numero,ano,nome_proposicao,id_proposicao,id_proposicao_principal,nome_proposicao_origem,tipo_proposicao,tema,ementa,...,partido_autor,data_apresentacao,regime_tramitacao,data_ultimo_despacho,ultimo_despacho,apreciacao,indexacao,situacao,link_inteiro_teor,apensadas
0,REQ,1030,2015,REQ 1030/2015 => PL 644/2015,1049075,994678.0,NaN,Requerimento de Urgência (Art. 155 do RICD),NaN,"Com base no art. 155 do Regimento Interno, req...",...,PP,18/03/2015,.,NaN,NaN,.,NaN,MESA - Tramitação do Requerimento Finalizada,http://www.camara.gov.br/proposicoesWeb/prop_m...,NaN
1,REQ,1041,2015,REQ 1041/2015 => PL 6232/2013,1049167,589924.0,NaN,Requerimento de Urgência (Art. 155 do RICD),NaN,"Requeremos a Vossa Excelência, nos termos do a...",...,PMDB,18/03/2015,.,NaN,NaN,.,NaN,COAPP(SGM) - Aguardando Encaminhamento,http://www.camara.gov.br/proposicoesWeb/prop_m...,NaN
2,PL,846,2015,PL 846/2015,1049380,NaN,NaN,Projeto de Lei,Direito Penal e Processual Penal,Acrescenta parágrafo ao artigo 121 do Decreto-...,...,PMDB,19/03/2015,Urgência art. 155 RICD,19/03/2015,Apense-se à(ao) PL-3131/2008. Proposição Sujei...,Proposição Sujeita à Apreciação do Plenário,"Alteração, Código Penal, agravação penal, homi...",Tranformada no(a) Lei Ordinária 13142/2015,http://www.camara.gov.br/proposicoesWeb/prop_m...,NaN
3,MPV,671,2015,MPV 671/2015,1050558,NaN,NaN,Medida Provisória,Desporto e Lazer; Tributação,Institui o Programa de Modernização da Gestão ...,...,NaN,20/03/2015,Urgência,01/07/2015,Ao Plenário para leitura. Publique-se.,Proposição Sujeita à Apreciação do Plenário,"Alteração, Estatuto de Defesa do Torcedor, Lei...",Tranformada no(a) Lei Ordinária 13155/2015,http://www.camara.gov.br/proposicoesWeb/prop_m...,NaN
4,PL,863,2015,PL 863/2015,1050643,NaN,NaN,Projeto de Lei,Tributação,"Altera a Lei nº 12.546, de 14 de dezembro de 2...",...,NaN,20/03/2015,Urgência art. 64 CF,13/05/2015,Revejo o despacho de deferimento proferido no ...,Proposição Sujeita à Apreciação do Plenário,"Alteração, Lei do Bem, contribuição previdenci...",Tranformada no(a) Lei Ordinária 13161/2015,http://www.camara.gov.br/proposicoesWeb/prop_m...,NaN


## Cruzando as duas tabelas

Vamos identificar se existe algum tipo de relação entre as duas tabelas(id do deputado, nome, ou algo do tipo)

In [25]:
from unicodedata import normalize
def remover_acentos(txt, codif='utf-8'):
    return normalize('NFKD', txt.decode(codif)).encode('ASCII','ignore')

In [26]:
proposicoes_df.columns

Index([u'tipo', u'numero', u'ano', u'nome_proposicao', u'id_proposicao',
       u'id_proposicao_principal', u'nome_proposicao_origem',
       u'tipo_proposicao', u'tema', u'ementa', u'explicacao_ementa', u'autor',
       u'ide_cadastro', u'uf_autor', u'partido_autor', u'data_apresentacao',
       u'regime_tramitacao', u'data_ultimo_despacho', u'ultimo_despacho',
       u'apreciacao', u'indexacao', u'situacao', u'link_inteiro_teor',
       u'apensadas'],
      dtype='object')

In [27]:
proposicoes_df['autor'].head(10)

0     Eduardo da Fonte
1    Leonardo Picciani
2    Leonardo Picciani
3      Poder Executivo
4      Poder Executivo
5    Leonardo Picciani
6       Mendonça Filho
7     Celso Russomanno
8     Eduardo da Fonte
9     Eduardo da Fonte
Name: autor, dtype: object

In [28]:
deputados_df.columns

Index([u'anexo', u'codOrcamento', u'condicao', u'email', u'fone', u'gabinete',
       u'idParlamentar', u'ideCadastro', u'matricula', u'nome',
       u'nomeParlamentar', u'partido', u'sexo', u'uf', u'urlFoto'],
      dtype='object')

In [29]:
deputados_df.head()

,anexo,codOrcamento,condicao,email,fone,gabinete,idParlamentar,ideCadastro,matricula,nome,nomeParlamentar,partido,sexo,uf,urlFoto
0,4,3283,Titular,dep.goulart@camara.gov.br,3215-5533,533,5830382,178980,361,ANTONIO GOULART DOS REIS,GOULART,PSD,masculino,SP,http://www.camara.gov.br/internet/deputado/ban...
1,4,2152,Titular,dep.betofaro@camara.gov.br,3215-5723,723,5830714,141335,19,JOSÉ ROBERTO OLIVEIRA FARO,BETO FARO,PT,masculino,PA,http://www.camara.gov.br/internet/deputado/ban...
2,4,2488,Titular,dep.rodrigodecastro@camara.gov.br,3215-5701,701,5830453,141531,268,RODRIGO BATISTA DE CASTRO,RODRIGO DE CASTRO,PSDB,masculino,MG,http://www.camara.gov.br/internet/deputado/ban...
3,4,2551,Titular,dep.laertebessa@camara.gov.br,3215-5340,340,5830785,141478,412,LAERTE RODRIGUES DE BESSA,LAERTE BESSA,PR,masculino,DF,http://www.camara.gov.br/internet/deputado/ban...
4,3,3712,Titular,dep.majorolimpio@camara.gov.br,3215-5279,279,5830405,178826,368,SERGIO OLIMPIO GOMES,MAJOR OLIMPIO,PDT,masculino,SP,http://www.camara.gov.br/internet/deputado/ban...


## Encontramos uma relação!
> O nome do autor no campo *autor* na tabela **proposicoes** se colocado no modo *uppercase* corresponde ao campo *nomeParlamentar* na tabela **deputados**, se o mesmo não estiver, é pq o referido deputado não está mais em exercício.

In [30]:
autor_proposicao = proposicoes_df.iloc[7]['autor']
print(autor_proposicao)
condicao = deputados_df['nomeParlamentar'] == autor_proposicao.upper()
deputados_df[condicao]


Celso Russomanno


,anexo,codOrcamento,condicao,email,fone,gabinete,idParlamentar,ideCadastro,matricula,nome,nomeParlamentar,partido,sexo,uf,urlFoto
471,4,3160,Titular,dep.celsorussomanno@camara.gov.br,3215-5960,960,5830790,73441,347,CELSO UBIRAJARA RUSSOMANNO,CELSO RUSSOMANNO,PRB,masculino,SP,http://www.camara.gov.br/internet/deputado/ban...


In [91]:
def formatar_nome(txt):
    result = []
    for t in txt.split():
            if len(t) > 2:           
                t = t.capitalize()
                result.append(t)
                continue

            result.append(t)
            
    return " ".join(result)

def attach_deputados_to_proposicoes():
    nome_parlamentar_normalizado = []
    
    for nomes in deputados_df['nomeParlamentar']:
        nome_parlamentar_normalizado.append(remover_acentos(nomes))
    
    deputados = [formatar_nome(x) for x in nome_parlamentar_normalizado]
    
    return proposicoes_df[proposicoes_df['autor'].isin(deputados)]

In [92]:
proposicoes_by_deputados = attach_deputados_to_proposicoes()

## Esse dataset(*by nazareno*) não nos parece um tanto viável para uso, por isso, será utilizado a API diretamente da câmara legislativa pelo fato da mesma já conter a relação entre o deputado e a proposição

> Veja o notebook *eda_proposicoes_PL*